In [1]:
import torch 
import numpy as np 
import os
import re 
import random 

import seaborn as sns
import matplotlib.pyplot as plt 

from transformers import AutoTokenizer
from torch.utils.data import Dataset 
from data import TranscriptDataset

In [2]:
ts = TranscriptDataset(tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased', 
                                                               overwrite=True))

In [6]:
ts[1]

{'input': {'input_ids': tensor([ 101, 3100, 2085, 2059, 2017, 2657, 1996, 3160, 2054,  102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
 'output': {'input_ids': tensor([2054, 2079, 2017, 2066, 2000, 2079, 2005, 4426, 1999,  102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}}

In [34]:
total_count = len(ts)
output_1 = sum([(out['output']['input_ids'] == 102).any(dim=0).float() for out in ts]).item()

In [37]:
total_count / output_1

1.3554028970275378